<img src="./images/DLI_Header.png" style="width: 400px;">

# 최종 평가

## 개요

이번 교육 과정 평가의 목표는 거대 모델을 구축하고 실행할 수 있는 능력을 평가하는 것입니다. 기존 코드를 DeepSpeed로 이식하고 액티베이션 체크 포인팅, 혼합 정밀도 훈련, ZeRo 리던던시 옵티마이저 등 다양한 DeepSpeed 기능을 사용할 수 있도록 일련의 구성 파일을 활용해 봅니다.

태스크를 포함할 수 있도록 단순화된 코드 베이스인 [minGPT](https://github.com/karpathy/minGPT) 를 의도적으로 선택했습니다. 본 코드는 최대 성능을 제공하지는 않지만 대표적이며 비교적 짧은 시간 내에 코딩 평가를 완료할 수 있도록 하는 Transformer의 최소 구현 형태입니다.

이 과제에서 우리는 또 다른 모델인 비전 트랜스포머 모델(ViT)에 대해서도 살펴볼 것입니다. 평가 과제를 진행하기 전에 [코드 예제](minGPT/minGPT/play_image.ipynb)를 미리 검토하십시오. 위의 코드 예제를 자유롭게 실행해 보십시오. 하지만 수렴를 위한 훈련은 상당한 시간이 소요되므로 이 과정을 조기에 끝내고 아래에서 논의하는 코드 마이그레이션에 집중하는 것이 도움이 될 수 있습니다.

## 소개

개념적으로 우리의 과제 목표는 다음과 같습니다:
- 학습 파이프라인의 독립 실행형 파이토치 구현을 DeepSpeed로 마이그레이션하고 "2개의 서버" 클러스터에서 효과적으로 훈련하십시오.
- 혼합 정밀도 훈련, 액티베이션 체크 포인팅, 제로 리던던시 옵티마이저와 같은 메모리 절약을 가능하게 하는 기능을 활성화합니다.
- 학습 중인 모델의 크기를 늘려봅니다.

아래 노트북은 프로세스를 안내하고 테스트 코드를 제공하여 올바른 해결 방법을 모색하는 데 도움이 됩니다. 평가가 끝날 때, 코딩을 완료하면 주피터랩 플랫폼으로 돌아가 'Assess' 버튼을 누릅니다. 버튼을 누르면 코드 파일과 DeepSpeed 구성 파일을 로드하고 이를 실행하여 구현의 정확성을 평가하는 자동화 프로세스가 실행될 예정입니다. 완료하는 데 몇 분 정도 걸릴 수 있으므로 이 단계를 실행할 충분한 시간을 두는 것이 좋습니다. 시간이 촉박한 경우 나중에 완료할 수 있도록 수정한 파일을 다운로드해두는 것이 좋습니다.

## 1단계: 베이스라인 구현

평가의 시작 지점인 [runStartingPoint.py](./minGPT/minGPT/runStartingPoint.py)부터 살펴보겠습니다. 이 파일은 이전에 검토했던 내용과 동일한 코드이며, 배치 실행을 위해 파이썬 파일로 추출되었습니다. 독립 실행형 모드에서 작동하는지 테스트해 보겠습니다. 다시 한 번 말하지만, 수렴을 위한 훈련에는 상당한 시간이 소요되므로, 일단 훈련 진행되는 것이 확인되면 자유롭게 훈련 과정을 중단하고 다음 단계로 넘어갑니다.

In [5]:
!python minGPT/minGPT/runStartingPoint.py

Files already downloaded and verified
Files already downloaded and verified
50000 10000
^C
Traceback (most recent call last):
  File "minGPT/minGPT/runStartingPoint.py", line 42, in <module>
    px = torch.cat([pluck_rgb(x) for x, y in train_data], dim=0).float()
  File "minGPT/minGPT/runStartingPoint.py", line 42, in <listcomp>
    px = torch.cat([pluck_rgb(x) for x, y in train_data], dim=0).float()
  File "minGPT/minGPT/runStartingPoint.py", line 41, in <lambda>
    pluck_rgb = lambda x: torch.from_numpy(np.array(x)).view(32*32, 3)[torch.randperm(32*32)[:5], :]
  File "/opt/conda/lib/python3.8/site-packages/PIL/Image.py", line 532, in __getattr__
    def __getattr__(self, name):
KeyboardInterrupt


## 2단계 : DeepSpeed 활성화

먼저 코드 변경을 최소화하고 DeepSpeed 라이브러리를 사용하도록 이전 훈련 스크립트를 수정하는 것으로 시작하겠습니다. 이 작업을 위해서는 다음이 필요합니다:

&nbsp; &nbsp; 1.  [runFirstDeepSpeed.py](./minGPT/minGPT/runFirstDeepSpeed.py)  에서 관련되는 부분을 수정합니다.   
&nbsp; &nbsp;  2.  [trainer.py](./minGPT/minGPT/mingpt/trainer.py) 에서 관련되는 부분을 수정합니다.   
&nbsp; &nbsp;  3.  DeepSpeed 구성 파일 `ds_config_basic.json`   을 생성합니다.   
&nbsp; &nbsp;  4. `deepspeed` 명령어를 사용하여 트레이닝을 실행합니다.


### 1. `runFirstDeepSpeed.py` 파일의 "ToDo Step 2" 섹션을 수정합니다.
[runFirstDeepSpeed.py](./minGPT/minGPT/runFirstDeepSpeed.py) 파일을 열고 DeepSpeed에 코드를 이식할 "ToDo Step 2" 섹션을 정의합니다. 정의해야 할 섹션은 총 4개입니다.

### 2.  `trainer.py` 파일에서 "ToDo Step 2" 섹션을 수정합니다.
[trainer.py](./minGPT/minGPT/mingpt/trainer.py) 파일을 열고 "ToDo Step 2" 섹션을 정의하여 `DeepSpeedTrainer` 클래스를 구현합니다. 수정/구현할 섹션은 총 6개입니다.

### 3. DeepSpeed 구성 파일 `ds_config_basic.json`을 생성합니다.
다음 셀에서 `FIXME`를 다음과 같이 수정합니다.
- GPU당 마이크로 배치 크기는 8입니다.
- Adam optimizer를 활성화하고 원래 코드 [runStartingPoint.py](./minGPT/minGPT/runStartingPoint.py)에서 학습률을 복사해야 합니다.
- 그래디언트 클리핑을 원래 코드  [runStartingPoint.py](./minGPT/minGPT/runStartingPoint.py)에 사용된 값으로 설정합니다.

In [101]:
%%writefile ./minGPT/minGPT/ds_config_basic.json
{
  "train_micro_batch_size_per_gpu": 8,
  "optimizer": {
    "type": "Adam",
    "params": {
      "lr": 3e-4
    }
  },
  "gradient_clipping": 1.0
}

Overwriting ./minGPT/minGPT/ds_config_basic.json


### 4.`deepspeed` 명령어로 트레이닝을 실행합니다.

다음 명령어는 4개의 GPU 트레이닝을 진행해야 하며 훈련이 진행되는 것을 확인해야 합니다. 다시 한 번 말하지만, 이 연습의 목표는 이 모델을 수렴하도록 훈련시키는 것이 아닙니다. 훈련이 실행 중이면 실행을 중단하고 다음 단계로 이동할 수 있습니다.

In [106]:
!deepspeed minGPT/minGPT/runFirstDeepSpeed.py --deepspeed --deepspeed_config minGPT/minGPT/ds_config_basic.json

[2022-12-02 09:07:40,731] [WARNING] [runner.py:159:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2022-12-02 09:07:40,899] [INFO] [runner.py:457:main] cmd = /opt/conda/bin/python3.8 -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMCwgMSwgMiwgM119 --master_addr=127.0.0.1 --master_port=29500 minGPT/minGPT/runFirstDeepSpeed.py --deepspeed --deepspeed_config minGPT/minGPT/ds_config_basic.json
[2022-12-02 09:07:42,456] [INFO] [launch.py:96:main] 0 NCCL_VERSION=2.11.4
[2022-12-02 09:07:42,456] [INFO] [launch.py:103:main] WORLD INFO DICT: {'localhost': [0, 1, 2, 3]}
[2022-12-02 09:07:42,456] [INFO] [launch.py:109:main] nnodes=1, num_local_procs=4, node_rank=0
[2022-12-02 09:07:42,456] [INFO] [launch.py:122:main] global_rank_mapping=defaultdict(<class 'list'>, {'localhost': [0, 1, 2, 3]})
[2022-12-02 09:07:42,456] [INFO] [launch.py:123:main] dist_world_size=4
[2022-12-02 09:07:42,456] [INFO] [launch.py:125:main] Setting CUDA_VIS

## 3단계: 멀티 노드 실행

위 코드는 이 특정 노드에 대해 4개의 GPU에서 실행되었지만, 우리의 목표는 수업 초반에 사용한 두개의 노드에서 작동하도록 만드는 것입니다. 앞서 작업한 코드를 다시 사용하여 위의 두개의 노드에서 실행되도록 작업 코드를 수정하세요. 적절한 Shell 스크립트를 생성하는 것부터 시작하겠습니다.

In [107]:
%%writefile ./minGPT/minGPT/runSlurmStep3.sh
#!/bin/bash
#SBATCH --job-name=dli_assessment_step3
#SBATCH --nodes=2
#SBATCH --ntasks-per-node=1       
#SBATCH --cpus-per-task=32 ### Number of threads per task (OMP threads)
#SBATCH -o /dli/megatron/logs/%j.out
#SBATCH -e /dli/megatron/logs/%j.err

# Number of nodes
NUM_NODES=2
# Number of GPUs per node
NUM_GPUS=2

deepspeed --num_nodes=${NUM_NODES} --hostfile /dli/minGPT/minGPT/hostfile --num_gpus=${NUM_GPUS} /dli/minGPT/minGPT/runFirstDeepSpeed.py \
    --deepspeed \
    --deepspeed_config minGPT/minGPT/ds_config_basic.json #FIXEME

Overwriting ./minGPT/minGPT/runSlurmStep3.sh


멀티 노드 실행을 활성화하려면 아래를 수정하십시오. 아래 명령어를 사용하여 멀티 노드 작업을 실행하십시오 (평가 제출 시 사용할 명령어이므로 파일 이름이나 경로를 변경하지 마십시오).

In [108]:
!sbatch ./minGPT/minGPT/runSlurmStep3.sh
!squeue

Submitted batch job 12
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
                12  slurmpar dli_asse    admin  R       0:01      2 slurmnode[1-2]


위 명령어가 실행되면 아래 명령어로 출력 및 오류 로그를 볼 수 있습니다. 작업 ID를 아래 명령어로 복사하십시오. 다시 한 번 코드 배포 시 아래 파일 위치에 다음과 같은 파일명 구조로 로그를 남기는 지 확인하십시오. 동일하게 평가 제출 시에도 해당 로그 파일들에 대한 확인이 필요합니다.

In [115]:
!JOB_ID=12;cat /dli/megatron/logs/$JOB_ID.out

[2022-12-02 09:08:50,536] [INFO] [runner.py:378:main] Using IP address of 172.18.0.9 for node slurmnode1
[2022-12-02 09:08:50,537] [INFO] [multinode_runner.py:65:get_cmd] Running on the following workers: slurmnode1,slurmnode2
[2022-12-02 09:08:50,537] [INFO] [runner.py:457:main] cmd = pdsh -f 1024 -w slurmnode1,slurmnode2 export NCCL_VERSION=2.11.4; export PYTHONPATH=/dli:/etc/assessment/; export PYTHONIOENCODING=utf-8;  cd /dli; /opt/conda/bin/python3.8 -u -m deepspeed.launcher.launch --world_info=eyJzbHVybW5vZGUxIjogWzAsIDFdLCAic2x1cm1ub2RlMiI6IFswLCAxXX0= --node_rank=%n --master_addr=172.18.0.9 --master_port=29500 /dli/minGPT/minGPT/runFirstDeepSpeed.py --deepspeed --deepspeed_config 'minGPT/minGPT/ds_config_basic.json'
slurmnode1: [2022-12-02 09:08:52,844] [INFO] [launch.py:96:main] 0 NCCL_VERSION=2.11.4
slurmnode1: [2022-12-02 09:08:52,844] [INFO] [launch.py:103:main] WORLD INFO DICT: {'slurmnode1': [0, 1], 'slurmnode2': [0, 1]}
slurmnode1: [2022-12-02 09:08:52,844] [INFO] [launc

In [116]:
!JOB_ID=12;cat /dli/megatron/logs/$JOB_ID.err

slurmnode1: 12/02/2022 09:09:29 - INFO - mingpt.model -   number of parameters: 1.000166e+07
slurmnode1: 12/02/2022 09:09:30 - INFO - mingpt.model -   number of parameters: 1.000166e+07
slurmnode2: 12/02/2022 09:09:30 - INFO - mingpt.model -   number of parameters: 1.000166e+07
slurmnode2: 12/02/2022 09:09:31 - INFO - mingpt.model -   number of parameters: 1.000166e+07
slurmnode1: 12/02/2022 09:09:32 - INFO - torch.distributed.distributed_c10d -   Added key: store_based_barrier_key:2 to store for rank: 1
slurmnode1: 12/02/2022 09:09:33 - INFO - torch.distributed.distributed_c10d -   Added key: store_based_barrier_key:2 to store for rank: 0
slurmnode2: 12/02/2022 09:09:33 - INFO - torch.distributed.distributed_c10d -   Added key: store_based_barrier_key:2 to store for rank: 2
slurmnode2: 12/02/2022 09:09:33 - INFO - torch.distributed.distributed_c10d -   Added key: store_based_barrier_key:2 to store for rank: 3
slurmnode2: 12/02/2022 09:09:33 - INFO - torch.distributed.distributed_c10d 

코드가 만족스러우면 다음 단계로 이동하기 전에 배치 작업이 종료되었는지 확인하십시오.

In [117]:
!squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
                12  slurmpar dli_asse    admin  R       1:12      2 slurmnode[1-2]


In [118]:
!scancel 12 #PASTE_JOB_ID_HERE

## 4단계: 코드를 추가로 개선하기

현재 우리는 활성화 체크포인팅을 수행할 수 있는 기능이 없습니다. 이번 단계에서는 DeepSpeed 라이브러리로 활성화 체크포인팅을 실행할 수 있는 코드를 소개합니다.

&nbsp; &nbsp;  1. 활성화 체크포인팅을 위한 트랜스포머 블록을 정의합니다.   
&nbsp; &nbsp;  2. 활성화 체크포인팅 및 FP16 트레이닝을 지원하는 DeepSeed 구성 파일을 만듭니다.   
&nbsp; &nbsp;  3.  sbatch 트레이닝 파일을 만들고 실행합니다.

### 1. 활성화 체크포인팅을 위한 트랜스포머 블록을 정의합니다.

DeepSpeed를 사용하여 모델(또는 모델의 일부)의 활성화 체크포인팅을 활성화하려면 포워드 패스 정의에서 각 블록을 `deepspeed.checkpointing.checkpoint()` ([더 알아보기](https://deepspeed.readthedocs.io/en/stable/activation-checkpointing.html#deepspeed.checkpointing.checkpoint)) 함수로 감싸야 합니다.

아래 예제는 CNN 블록 2개와 DeepSpeed로 활성화 체크포인팅을 위해CNN 블록을 감싸는 선형 레이어가 있는 간단한 합성곱 신경망 정의를 보여줍니다.

```
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn_block_1 = nn.Sequential(*[nn.Conv2d(3, 32, 3, padding=1),nn.ReLU(),nn.MaxPool2d(kernel_size=2)])
        self.cnn_block_2 = nn.Sequential(*[nn.Conv2d(64, 64, 3, padding=1),nn.ReLU(),nn.MaxPool2d(kernel_size=2)])
        self.flatten = lambda inp: torch.flatten(inp, 1)
        self.linearize = nn.Sequential(*[ nn.Linear(64 * 8 * 8, 512),nn.ReLU()])
        self.out = nn.Linear(512, 10)
    
    def forward(self, X):
        X = deepspeed.checkpointing.checkpoint(self.cnn_block_1, X)
        X = deepspeed.checkpointing.checkpoint(self.cnn_block_2, X)
        X = self.flatten(X)
        X = self.linearize(X)
        X = self.out(X)
        return X

```

비슷한 메커니즘은 `torch.utils.checkpoint.checkpoint()`기능을 통해 토치와 함께 구현됩니다.


우리의 경우, Vision Transformer 모델은 `./minGPT/minGPT/mingpt/model.py` 파일의 GPT 클래스로 구현됩니다. DeepSpeed 활성화 체크포인팅으로 트랜스포머 블록을 감싸야 합니다.  [model.py](./minGPT/minGPT/mingpt/model.py 파일에서 "Step 4 ToDo" 태스크를 수정합니다.

### 2. DeepSeed 구성 파일을 만듭니다.

시작하기 전에 config-json 파일의 DeepSpeed 문서에서 [activation-checkpointing](https://www.deepspeed.ai/docs/config-json/#activation-checkpointing) 을 확인해볼 수 있습니다.

아래 셀의 `#FIXME`를 다음과 같이 수정하여 `ds_config_step4.json` 을 만듭니다.
- 활성화 체크포인팅을 활성화합니다.
- 활성화 체크포인팅이 제대로 작동하는지 확인하려면 GPU당 마이크로 배치 크기를 128로 설정하십시오.
- 체크포인트 수를 12개로 만듭니다.
- FP16 트레이닝을 활성화합니다.

In [1]:
%%writefile minGPT/minGPT/ds_config_step4.json
{
  "train_micro_batch_size_per_gpu": 128,
  "optimizer": {
    "type": "Adam",
    "params": {
      "lr": 3e-4
    }
  },
  "gradient_clipping": 1.0,
    "activation_checkpointing": {
        "partition_activations": true,
        "number_checkpoints": 12,
        "cpu_checkpointing": true
    },
    "fp16": {
        "enabled": true,
        "auto_cast": false
    }
}

Overwriting minGPT/minGPT/ds_config_step4.json


### 3. sbatch 트레이닝 파일을 실행합니다.


먼저 훈련용 파이썬 스크립트`runFirstDeepSpeed.py`의 복사본을 만들어 보겠습니다.

In [2]:
!cp /dli/minGPT/minGPT/runFirstDeepSpeed.py /dli/minGPT/minGPT/runStep4.py

이제 sbatch 파일 `runSlurmStep4.sh`을 만들어 보겠습니다.

In [3]:
%%writefile ./minGPT/minGPT/runSlurmStep4.sh
#!/bin/bash
#SBATCH --job-name=dli_assessment_step4
#SBATCH --nodes=2
#SBATCH --ntasks-per-node=1       
#SBATCH --cpus-per-task=32 ### Number of threads per task (OMP threads)
#SBATCH -o /dli/megatron/logs/%j.out
#SBATCH -e /dli/megatron/logs/%j.err

# Number of nodes
NUM_NODES=2
# Number of GPUs per node
NUM_GPUS=2

deepspeed --num_nodes=${NUM_NODES} --hostfile /dli/minGPT/minGPT/hostfile --num_gpus=${NUM_GPUS} /dli/minGPT/minGPT/runStep4.py \
    --deepspeed \
    --deepspeed_config /dli/minGPT/minGPT/ds_config_step4.json

Overwriting ./minGPT/minGPT/runSlurmStep4.sh


위의 작업을 완료한 후 아래 명령어를 실행하여 훈련 작업을 슬럼 스케줄러에 제출하십시오.

In [4]:
!sbatch /dli/minGPT/minGPT/runSlurmStep4.sh
!squeue

Submitted batch job 15
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
                15  slurmpar dli_asse    admin PD       0:00      2 (None)
                14  slurmpar dli_asse    admin  R       1:11      2 slurmnode[1-2]


아래를 사용하여 코드 실행을 확인합니다 (배치 크기가 크더라도 진행이 확인되어야 합니다).

In [11]:
!JOB_ID=15;cat /dli/megatron/logs/$JOB_ID.out

[2022-12-02 09:14:05,376] [INFO] [runner.py:378:main] Using IP address of 172.18.0.9 for node slurmnode1
[2022-12-02 09:14:05,377] [INFO] [multinode_runner.py:65:get_cmd] Running on the following workers: slurmnode1,slurmnode2
[2022-12-02 09:14:05,377] [INFO] [runner.py:457:main] cmd = pdsh -f 1024 -w slurmnode1,slurmnode2 export NCCL_VERSION=2.11.4; export PYTHONPATH=/dli:/etc/assessment/; export PYTHONIOENCODING=utf-8;  cd /dli; /opt/conda/bin/python3.8 -u -m deepspeed.launcher.launch --world_info=eyJzbHVybW5vZGUxIjogWzAsIDFdLCAic2x1cm1ub2RlMiI6IFswLCAxXX0= --node_rank=%n --master_addr=172.18.0.9 --master_port=29500 /dli/minGPT/minGPT/runStep4.py --deepspeed --deepspeed_config '/dli/minGPT/minGPT/ds_config_step4.json'
slurmnode2: [2022-12-02 09:14:07,690] [INFO] [launch.py:96:main] 1 NCCL_VERSION=2.11.4
slurmnode2: [2022-12-02 09:14:07,690] [INFO] [launch.py:103:main] WORLD INFO DICT: {'slurmnode1': [0, 1], 'slurmnode2': [0, 1]}
slurmnode2: [2022-12-02 09:14:07,690] [INFO] [launch.py

In [12]:
!JOB_ID=15;cat /dli/megatron/logs/$JOB_ID.err

slurmnode1: Traceback (most recent call last):
slurmnode1:   File "/dli/minGPT/minGPT/runStep4.py", line 36, in <module>
slurmnode1:     deepspeed.init_distributed()
slurmnode1:   File "/opt/conda/lib/python3.8/site-packages/deepspeed/utils/distributed.py", line 51, in init_distributed
slurmnode1:     torch.distributed.init_process_group(backend=dist_backend,
slurmnode1:   File "/opt/conda/lib/python3.8/site-packages/torch/distributed/distributed_c10d.py", line 578, in init_process_group
slurmnode1:     store, rank, world_size = next(rendezvous_iterator)
slurmnode1:   File "/opt/conda/lib/python3.8/site-packages/torch/distributed/rendezvous.py", line 229, in _env_rendezvous_handler
slurmnode1:     store = _create_c10d_store(master_addr, master_port, rank, world_size, timeout)
slurmnode1:   File "/opt/conda/lib/python3.8/site-packages/torch/distributed/rendezvous.py", line 157, in _create_c10d_store
slurmnode1:     return TCPStore(
slurmnode1: RuntimeError: Address already in use
pdsh@s

코드 실행이 만족스럽다면 다음 단계 이동을 위해 배치 작업의 실행을 취소하는 것을 잊지 마십시오.

In [10]:
!squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
                15  slurmpar dli_asse    admin  R       0:03      2 slurmnode[1-2]


In [9]:
!scancel 14 #PASTE_JOB_ID_HERE

### 추가적인 최적화 고려 사항

훈련 과정에 참여하는 모든 워커는 동일한 출력을 생성합니다. 따라서 k-mean 값은 두 번 계산됩니다. 
k-mean 구현을 조정하여 한 번만 실행하고 결과를 모든 워커에 재배포할 수 있습니다. 
다음은 어떻게 실행할 수 있는 지에 대한 예시입니다.

```import torch.distributed as dist
def run_kmeans(x, ncluster, niter=8, rank, size):
    print('KMeans executed on rank ', rank, ' Worlds size ', size)
    N, D = x.size()
    c = x[torch.randperm(N)[:ncluster]] # init clusters at random
    c = c.cuda(args.local_rank) # move the tensor to the GPU for exchange
    if rank == 0:
        # Computing KMeans only on rank 0 
        with torch.no_grad():
            c = kmeans(x, ncluster, niter)
    # We now have computed the clusters so can proceed to the exchange
    dist.barrier()
    print('Broadcasting')
    dist.broadcast(C.cuda(args.local_rank), src=0)
    c=c.cpu()
    print('Rank ', rank, ' has data ', C.size())
    return c

C=run_kmeans(px, ncluster, niter=8, dist.get_rank(), dist.get_world_size())    
```


## 5단계: 스케일업

이제 최소한의 기능이 구현되었으므로 훈련 작업을 확장해 보겠습니다. 평가에서 스케일업 부분은 모델을 상당히 크게 만들 것입니다. 

&nbsp; &nbsp; 1. 모델의 아키텍처를 확장합니다.   
&nbsp; &nbsp; 2. 활성화 체크포인팅, FP16 트레이닝, ZeRO 옵티마이저를 활성화하는 DeepSeed 구성 파일을 만듭니다.     
&nbsp; &nbsp; 3. sbatch 훈련 파일을 만들고 실행합니다.  

### 1. 모델의 아키텍처를 확장합니다.
트레이닝 스크립트를 수정하기 전에 수정할 복사본을 만드는 것부터 시작하겠습니다:

In [27]:
!cp /dli/minGPT/minGPT/runFirstDeepSpeed.py /dli/minGPT/minGPT/runStep5.py

 [runStep5.py](./minGPT/minGPT/runStep5.py)에서  "GPTConfig" 섹션을 수정하여 VisionTransformers의 레이어 수를 **24** 로 조정합니다.  "GPTConfig" 에서는 신경망 차원의 아키텍처가 다음과 같이 정의됩니다.

```
mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size,
                  embd_pdrop=0.0, resid_pdrop=0.0, attn_pdrop=0.0,
                  n_layer=12, n_head=8, n_embd=256)
```



### 2. 활성화 체크포인팅, FP16 트레이닝, ZeRO 옵티마이저를 지원하는 DeepSeed 구성 파일을 만듭니다.

[ds_config_step5.json](./minGPT/minGPT/ds_config_step5.json) 을 변경하여 다음을 사용하도록 재구성합니다.:
- 그래디언트 누적 및 4단계 누적 단계를 실행하여 글로벌 배치 크기를 늘립니다 (고정된 하이퍼 파라미터를 유지하는 데 자주 필요합니다).
- 12개의 체크포인트가 아닌 24개의 체크포인트를 만들기 위한 활성화 체크포인팅을 실행합니다.
- FP16 트레이닝
- 파라미터 및 옵티마이저 상태 모두에 대해 CPU 오프로드를 지원하는 Zero Stage 3 옵티마이저를 실행합니다. 자세한 내용은  [ZeRO 설명서](https://deepspeed.readthedocs.io/en/latest/zero3.html) 를 참조하십시오.

In [13]:
%%writefile minGPT/minGPT/ds_config_step5.json
{
  "train_micro_batch_size_per_gpu": 128,
  "optimizer": {
    "type": "Adam",
    "params": {
      "lr": 3e-4
    }
  },
  "gradient_clipping": 1.0,
    "activation_checkpointing": {
        "partition_activations": true,
        "number_checkpoints": 24,
        "cpu_checkpointing": true
    },
    "fp16": {
        "enabled": true,
        "auto_cast": false
    },
    "zero_optimization": {
        "stage": 3,
    "offload_optimizer": {
        "device": "cpu"
    },
    "offload_params": {
        "device": "cpu"
    }
    }
}

 # FIXME enable activation_checkpointing  (v)
 # FIXME enable FP16 training (v)
 # FIXME enable zero_optimization stage 3 with CPU offload for both parameters and optimizer states  

Overwriting minGPT/minGPT/ds_config_step5.json


### 3. sbatch 트레이닝 파일을 만들고 실행합니다. 
다음 셀을 실행하여 5단계 훈련에 대한 sbatch 스크립트를 생성합니다.

In [14]:
%%writefile ./minGPT/minGPT/runSlurmStep5.sh
#!/bin/bash
#SBATCH --job-name=dli_assessment_step5
#SBATCH --nodes=2
#SBATCH --ntasks-per-node=1       
#SBATCH --cpus-per-task=32 ### Number of threads per task (OMP threads)
#SBATCH -o /dli/megatron/logs/%j.out
#SBATCH -e /dli/megatron/logs/%j.err

# Number of nodes
NUM_NODES=2
# Number of GPUs per node
NUM_GPUS=2

deepspeed --num_nodes=${NUM_NODES} --hostfile /dli/minGPT/minGPT/hostfile --num_gpus=${NUM_GPUS} /dli/minGPT/minGPT/runStep5.py \
    --deepspeed \
    --deepspeed_config /dli/minGPT/minGPT/ds_config_step5.json

Overwriting ./minGPT/minGPT/runSlurmStep5.sh


위의 내용을 변경한 후에는 다음 명령어를 사용하여 작업을 실행하십시오:

In [15]:
!sbatch /dli/minGPT/minGPT/runSlurmStep5.sh
!squeue

Submitted batch job 16
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
                16  slurmpar dli_asse    admin PD       0:00      2 (Resources)
                15  slurmpar dli_asse    admin  R       0:30      2 slurmnode[1-2]


다음을 사용하여 코드가 실행되었는지 확인합니다:



In [19]:
!JOB_ID=16;cat /dli/megatron/logs/$JOB_ID.out

[2022-12-02 09:14:44,540] [INFO] [runner.py:378:main] Using IP address of 172.18.0.9 for node slurmnode1
[2022-12-02 09:14:44,541] [INFO] [multinode_runner.py:65:get_cmd] Running on the following workers: slurmnode1,slurmnode2
[2022-12-02 09:14:44,541] [INFO] [runner.py:457:main] cmd = pdsh -f 1024 -w slurmnode1,slurmnode2 export NCCL_VERSION=2.11.4; export PYTHONPATH=/dli:/etc/assessment/; export PYTHONIOENCODING=utf-8;  cd /dli; /opt/conda/bin/python3.8 -u -m deepspeed.launcher.launch --world_info=eyJzbHVybW5vZGUxIjogWzAsIDFdLCAic2x1cm1ub2RlMiI6IFswLCAxXX0= --node_rank=%n --master_addr=172.18.0.9 --master_port=29500 /dli/minGPT/minGPT/runStep5.py --deepspeed --deepspeed_config '/dli/minGPT/minGPT/ds_config_step5.json'
slurmnode1: [2022-12-02 09:14:46,831] [INFO] [launch.py:96:main] 0 NCCL_VERSION=2.11.4
slurmnode1: [2022-12-02 09:14:46,831] [INFO] [launch.py:103:main] WORLD INFO DICT: {'slurmnode1': [0, 1], 'slurmnode2': [0, 1]}
slurmnode1: [2022-12-02 09:14:46,831] [INFO] [launch.py

In [20]:
!JOB_ID=16;cat /dli/megatron/logs/$JOB_ID.err

slurmnode1: Traceback (most recent call last):
slurmnode1:   File "/dli/minGPT/minGPT/runStep5.py", line 36, in <module>
slurmnode1:     deepspeed.init_distributed()
slurmnode1:   File "/opt/conda/lib/python3.8/site-packages/deepspeed/utils/distributed.py", line 51, in init_distributed
slurmnode1:     torch.distributed.init_process_group(backend=dist_backend,
slurmnode1:   File "/opt/conda/lib/python3.8/site-packages/torch/distributed/distributed_c10d.py", line 578, in init_process_group
slurmnode1:     store, rank, world_size = next(rendezvous_iterator)
slurmnode1:   File "/opt/conda/lib/python3.8/site-packages/torch/distributed/rendezvous.py", line 229, in _env_rendezvous_handler
slurmnode1:     store = _create_c10d_store(master_addr, master_port, rank, world_size, timeout)
slurmnode1:   File "/opt/conda/lib/python3.8/site-packages/torch/distributed/rendezvous.py", line 157, in _create_c10d_store
slurmnode1:     return TCPStore(
slurmnode1: RuntimeError: Address already in use
pdsh@s

다음 단계로 이동하기 전에 실행 및 보류 중인 모든 작업을 중지해야합니다. 아니면 평가 결과 Fail을 받을 것입니다!

In [21]:
!squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
                16  slurmpar dli_asse    admin  R       0:39      2 slurmnode[1-2]


In [16]:
!scancel 15 #PASTE_JOB_ID_HERE

## 6단계: 평가

위에 나열된 변경 사항을 모두 구현한 경우 5단계에서 확인한 작업 ID를 아래 코드 블록에 입력하십시오. 문제가 올바르게 완료된 경우 "Assessment Passed!" 메시지가 나타납니다. 행운을 빌겠습니다! :) 

In [43]:
from run_assessment import run_assessment
job_id = 5
run_assessment(job_id)

Evaluating Job ID: 5
Error: job did not complete a training step with the expected configuration. Please try again.


"Assessment Passed!"가 나타나면 DLI 포털로 돌아가서 평가(assess) 버튼을 누르시면 인증서가 생성됩니다. 성공적으로 과정을 마무리하셨습니다! 축하합니다!